# IPython para administración de sistemas Unix

Pueden listarse las funciones dentro de un módulo con la consola de `IPython` usando el **tabulador** cuando se importa el módulo, se escribe por consola y se añade un punto:

```python
import subprocess
subprocess.
```

Para conocer un ejemplo de uso de una de las funciones puede añadirse un `?` detrás de la función como en el ejemplo. Puede obtenerse más ayuda si se introducen dos `??`.

In [ ]:
subprocess.getoutput??

Es buena idea cuando se crean scripts de shell con Python envolver todas las funciones que representan un módulo en un `main` de tal manera que se ejecuten secuencialmente todas las funciones. De esta manera se tienen por un lado llamadas individuales a las funciones y por otro el conjunto de todas ellas. A esta función `main` se le denomina **flujo de control**.

In [ ]:
import subprocess

# Comando 1
def uname_func():
    
    uname = "uname"
    uname_arg = "-a"
    print("Gathering system informatiokn with %s command:\n" % uname)
    subprocess.call([uname, uname_arg])
    
    
#Comando 2
def disk_func():
    
    diskspace = "df"
    diskspace_arg = "-h"
    print("Gathering diskpace information %s command:\n" % diskspace)
    subprocess.call([diskspace, diskspace_arg])

    
# Función principal que invoca otras funciones.
def main():
    uname_func()
    disk_func()
    
main()

## Funciones mágicas

Para listar las funciones mágicas introducir: `lsmagic`. También puede utilizarse el tabulador si se pone por delante el caracter especial `%`. Puede desplegarse una ayuda si se introduce el comando: `magic`.

In [ ]:
lsmagic

Un ejemplo de una función mágica, permite crear `alias` como en el ejemplo:

```python
%alias d ls -F
```

Para desactivarlo, utilizar la función mágica `%unalias`.

In [ ]:
%alias d ls -F

Por ejemplo, pueden listarse las funciones y módulos interactivos que se están usando en la sesión de IPython con la **función mágica** `%who`o `%whos` con algo extra de información. Por ejemplo, hasta esta celdilla se han ejecutado otras celdas que contienen información. La lista se muestra en la siguiente celdilla.

In [ ]:
%whos

Una guía rápida de las funciones mágicas puede sacarse con `%quickref`. Un ejemplo se muestra a continuación.

In [ ]:
%quickref

Un ejemplo de alias útil para gestionar un sistema Unix pueden ser, como se dijo antes, los alias. Si, por ejemplo, se quiere utilizar el comando `netstat -plunt` de manera más rápida, puede hacerse u alias como el siguiente:

In [ ]:
%alias nss netstat -plunt

In [ ]:
nss

Además, puede utilizarse las metodologías **hacer-nada** o **hacer-todo** donde se le pueden pasar o no argumentos a un alias o comando e interpretarse de distinta manera. Teniendo por ejemplo el alias:

In [ ]:
%alias achoo echo first: "|%s|", second "|%s|"

In [ ]:
achoo foo bar

Para hacerlo con o sin argumentos:

In [ ]:
%alias achoo echo "|%l|"

In [ ]:
achoo

Otra forma de ejecutar un comando de shell es colocando el símbolo `!` delante. La salida de un comando de shell puede almacenarse en una lista, como por ejemplo.

In [ ]:
user = 'nachoaz'
process = 'bash'
l = !ps aux | grep $user | grep $process
l # Imprimir la variable 'l'.

## cd y marcadores 

## Bookmarks

Crear marcadores se puede hacer de diferentes maneras. Una de ellas es la siguiente: con la directiva `bookmark t` estando, por ejemplo, en el directorio `/tmp` se crea un marcador en ese directorio.

In [ ]:
cd /tmp

In [ ]:
bookmark t

Otra manera de crear un marcador requiere escribir un poco más.

In [ ]:
bookmark h /home/

Pueden listarse los marcadores con la siguiente opción. **Nota:** Los marcadores se conservan entre sesiones de IPython por lo que no es necesario cargarlos de nuevo.

In [ ]:
%bookmark -l

Para eliminar un marcador se utiliza el argumento `-d` como sigue:

In [ ]:
bookmark -d h

In [ ]:
bookmark -l

## dhist

El comando `dhist` no solamente guarda el histórico de la sesión actual sino de sesiones anteriores.

In [ ]:
dhist

Puede pasarsele argumentos para especificar, por ejemplo, la salida máxima del comando, haciendo una lista de los últimos 5 elementos de la historia con: `dhist 5` o un rango entre valores con `dhist 3 7` 

## Aplicación de Variable

Hasta ahora había una clara diferencia entre los comandos de shell e IPython pero vamos a ver que se pueden cruzar. Se va a tomar un valor que obtendremos de Python y pasarlo a la Shell. Aunque el ejemplo es "poco realista" dado que es raro crear 10 archivos con la fecha dentro, nos da una idea de la utilidad en el cruce de información entre ambos lenguajes.

In [ ]:
# {} = elemento actual de la lista
for i in range(3):
    !date > {i}.txt

In [ ]:
!ls -l

!cat 0.txt

Eliminamos los archivos creados en el apartado anterior.

In [ ]:
!rm *.txt

## Procesado de cadena

mOtra herramienta muy interesante que ofrece IPython es procesar en cadena el resultado de los comandos de shell del sistema. Si, por ejemplo, se quiere ver el PID de todos los procesos que pertenecen a un usuario se podría hacer de la siguiente manera: 

`ps aux | awk '{if ($1 == "myuser") print $2}'`

Si ahora intentamos repetir el proceso usando IPython hay que hacerlo en una pequeña secuencia que es igualmente legible.

In [ ]:
ps = !ps aux
ps.grep('mysuer', prune=False).fields(0, 1, 8)

El argumento `prune` se utiliza para que el retorno del argumento sea lo contrario a lo que se quiere o lo que se quiere. Por ejemplo, si quisieramos que el retorno fuera 'todos los usuarios que no sean el nuestro' (`root` por ejemplo) se pondría: `ps.grep('myuser', prune=True)`, es decir, __excluir__ del filtrado. De esa manera devuelve todos los usuarios que no sean `myuser`.

Con el método `fields()` se le especifican a `grep` las columnas que queremos devolver una vez se ejecute el comando. Hace las veces cuando se imprime una columna con `awk` utilizando `$1`, `$2`. En el ejemplo anterior, las columnas `0`, `1` y `8` hacen referencia a `USERNAME`, `PID` y `STARTTIME`.

Otro elemento muy interesante es, seguido del método `fields()` que va seguido del método `grep()` y que ambos devuelven objetos del mismo tipo, se puede añadir el elemento `s` que accede directamente a la lista en el proceso. Esto proporciona una cadena de PID separados por espacio con la que la Shell ya puede trabajar y de podría, por ejemplo hacer `kill $pids` desde dentro de IPython.

Por último, el método `grep()` adopta un último parámetro opcional denominado `field`. Si se especifica, los criterios de búsqueda tienen que coincidir con ese campo para que se incluya en el resultado. Esto coincidía con las dilas exactas que se quieren pero mostraba toda la fila. Para llegar solo al PID hay que añadirle `fields(1)` detrás.

In [ ]:
ps = !ps aux
ps.grep('nachoaz', field=0).fields(1)

## Perfil sh

Un perfil es simplemente un conjunto de datos de configuracioń que se carga cuando se inicia IPython. Puede personalizarse un número de perfiles para hacer qye IPython se lleve a cabo de diferentes formas dependiendo de las necesidades de la sesión. Para invocar un perfil específico se utiliza la opción de línea de comando `-p` y se especifica el perfil que se quiere utilizar.

El perfil `sh` es el más utilizado dado que incorpora ciertos elementos de configuración de modo que IPython se convierte en una shell del sistema más amigable.

Dos ejemplos de valores de configuración que son diferentes del perfil estándar de IPython son que `sh` muestra el directorio actual y `rehash` y su `PATH` de modo que tiene acceso instantáneo a todos los mismos ejecutables que tednría en `bash`.

`sh` permite además la extensión `envpersist`. Permite modificar varias variables de entorno fácilmente y de forma persistente para su perfil `sh` IPython sin necesidad de actualizar el `.bash_profile` o `.bashrc`.

En un terminal introducimos:

```bash
ipython --profile=sh

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.5.0 -- An enhanced Interactive Python. Type '?' for help.

IPython profile: sh

In [1]:                                                                                                           
```

Y tenemos la consola de IPython con el perfil de `sh` cargado.

## Recopilación de información

### page

Si un objeto no entra en la pantalla, con la función mágica `page` puede obtenerse un paginador. El predeterminado por el sistema es `less`. Un ejemplo sería el siguiente que si se compara con la función `page` permite tenerlo de manera completa en lugar de tener que hacer _scroll_ para ver todo el contenido en la celdilla.

In [ ]:
p = !ps aux
print(p)

In [ ]:
%page -r p

### pinfo

Proporciona información como tipo, clase base, espacio de nombre y _docstring_. Si tenemos un módulo que contiene una serie de clases se puede solicitar información del propio módulo.

In [ ]:
import time
import cv2

In [ ]:
%pinfo time

Puede soliitarse información acerca de:
- Del propio módulo:
    ```python
    pinfo module
    ```
- De una clase del módulo:
    ```python
    pinfo module.foo
    ```
- De una instancia de una de las clases:
    ```python
    f = module.Foo()
    pinfo p
    ```
`pinfo` tiene la misma funcionalidad que `?` delante del nombre de un objeto como se vió al principio.

### psearch

Esta función mágica buscará objetos Python por nombre con la ayuda de comodines. Declarando una secuencia de objetos:

In [ ]:
a = 1
aa = "one"
b = 2
bb = "two"
c = 3
cc = "three"

In [ ]:
psearch a*

Los objetos que se han encontrado además de `a`, `aa`, `b`, etc son objetos incorporados. Una alternativa rápida a `psearch` es cambiarlo por `?`. Puede usarse `-s` para buscar, `-e` para excluir en un determinado espacio de nombre. Estos espacios de nombre incluyen `builtin`, `user_local`, `user_global`, `internal` y `alias`. Por defecto, `psearch` busca en los dos primeros. Para buscar explícitaente en `user_local` puede pasar una opción a `psearch` para excluir buscar el espacio de nombre `builtin`. La ruta de búsqueda predeterminada para `psearch` es: `builtin` y `user_local` por lo que si especificamos `-s user` buscar `builtin` y `user_local` seguirá siendo lo que hemos pedido que haga.

In [ ]:
%psearch -e builtin a*

In [ ]:
%psearch -s user_local a*

`psearch` permite también buscar tipos específicos de objetos. Si, por ejemplo, se busca el espacio de nombres `user_local` en busca de enteros:

In [ ]:
%psearch a*.* int

Y si buscamos cadenas:

In [ ]:
%psearch -e builtin * string

### who

IPython proporciona un número de facilidades para listar todos los objetos interactivos. 

In [ ]:
%who

O especificando el tipo:

In [ ]:
#%who int
%who str

### who_ls

Es similar a `who` pero retorna una lista con la que se pueden hacer otras operaciones:

In [ ]:
%who_ls str

In [ ]:
# con _ se accede a la última lista de nombres.
# Es: el último resultado
for i in _:
    print(i)

### whos

Es similar a `who` pero imprime información que no saca la anterior función mágica.

In [ ]:
%whos

### macro

`macro` permite definir un bloque de código que se ejecutará más adelante en línea con cualquier código que se esté trabajando. Es diferente de crear funciones o métodos. La macro se hace consciente del contexto actual del código. Si tiene un conjunto común de pasos de procesado que frecuentemente se ejecuta en todos los archivos, se puede crear una `macro` para trabajar en los archivos. Por ejemplo

In [ ]:
dirlist = []

In [ ]:
for f in dirlist:
    print("Working on", f)
    print("done with", f)
    print("moving %s to %s.done" % (f, f))
    print("*" * 40)

Sintaxis: `macro nombre_macro rango_de_lineas`.

Donde el rango de líneas es la lista de las lineas de la historia que se quiere incorporar al `macro`.

_Nota: Hay que fijarse en el número que está encerrado en la celdilla, ya que el código puede editarse y guardarse después. Una modificación en la celdilla anterior eleva otro número de ejecución y ese será el que queramos guardar. De igual modo pueden establecerse rangos de celdillas separado por `-`_.

In [ ]:
%macro procdir 33

In [ ]:
dirlist = ['a.txt', 'b.txt', 'c.txt']

In [ ]:
procdir

# store

Para almacenar los macros guardados se utiliza la función `store` seguido del nombre que se le quiera dar a la _macro_.

In [ ]:
%store procdir

### reset

Para eliminar todas las variables del espacio de nombres se utiliza la función `reset`. En el siguiente ejemplo puede verse como se crean unas vatiables que se eliminan posteriormente utilizando `reset`.

In [ ]:
a = 1
b = 2
c = 3
%whos

In [ ]:
%reset

In [ ]:
%whos

### run

Con la función `run` se puede ejecutar un archivo que se especifique a IPython. Esto permite trabajar en un módulo Python en un editor de texto externo y comprobar interactivamente los cambios que se hace en él desde IPython o Jupyter como se está haciendo en este documento. Después de ejecutar el programa, se devuelve a la shell de IPython. La sintaxis para ejecutar es: `run opciones_archivo arguments`. Existen diferentes opciones:
 
- `-n`: hace que la variable `__name__` del módulo se establezca no en `__main__` sino en su propio nombre. Esto hace que el módulo se ejecute como si simplemente se importara.
- `-i`: Ejecuta el módulo en el espacio de nombre actual de IPython y proporciona al módulo acceso a todas las variables definidas.
- `-e`: Hace que IPython ignore llamadas a `sys.exit()` y excepciones `SystemExit`. Si algo de esto ocurre, IPython simplemente continúa con el programa.
- `-t`: Hace que IPython imprima información sobre la longitud de tiempo que tardó el módulo en ejecutarse.
- `-d`: hace que el módulo especificado se ejecuta bajo el depurador Python (pdb).
- `-p` : Ejecuta el módulo especificado bajo el creador de perfil Python.

### save

Para guardar las líneas de entrada especificadas en el archivo de salida se utiliza la sintaxis: 

```python
%save opciones nombre_archivo lineas
```

La única opción de la función `save` es `-r` que especifica que se guardará el archivo de entrada sin formato (raw) en lugar de traducido.

### rep

Esta función toma un número de parámetros que se podría encontrar de utilidad. Utilizar `rep` sin parámetros toma el último resultado queq se procesó (como el `ans` de las calculadoras científicas) y sitúa una representación de ello en la siguiente entrada.

In [ ]:
def format_str(s):
    return "str(%s)" % s

In [ ]:
format_str(1)

In [ ]:
%rep

In [ ]:
str(1)

## Texto

Métodos `lstrin()`, `rstrip()`. `strip()`

- `lstrip()`: Utilizarlo sin argumentos elimina todo espacio en blanco que esté al **principio** de la cadena y devuelve una nueva cadena (no modifica la original).
- `rstrip()`: Sin argumentos elimina todo espacio en blanco al **final** de la cadena y devuelve una nueva cadena.
- `strip()`: Elimina espacio en blanco al **principio** y **final** de la cadena y devuelve una nueva cadena.

Ejemplos:

In [ ]:
spacious_string = "\n\t Texto sin espacios \t\r"
print(spacious_string)

In [ ]:
print(spacious_string.lstrip()) # Elimina los espacios al principio de la cadena (preserva el \t\r)
print(spacious_string.rstrip()) # Elimina los espacios al final de la cadena (preserva el \n\t)
print(spacious_string.strip())  # Elimina los espacios al principio y al final de la cadena

Estos métodos con _strings_ también son útiles para eliminar partes de una cadena, como por ejemplo parsear un `xml`.

In [ ]:
##### xml_tag = "<some_tag>"
xml_tag.lstrip("<")

In [ ]:
xml_tag.lstrip(">") # Como al principio de la cadena no existe ese caracter, el resutado es el mismo

In [ ]:
xml_tag.rstrip(">")

In [ ]:
xml_tag.strip("<").strip(">")

In [ ]:
xml_tag.strip("<>")

In [ ]:
gt_it_str = "<><><>gt lt str<><><>"

In [ ]:
gt_it_str.strip("<>")

También hay que tener cuidado con `strip()` dado que elimina **todas** las coincidencias que encuentre en la cadena y no patrones como puede parecer. En el siguiente ejemplo puede verse claramente.

In [ ]:
foo_str = "<fooooooooo>hola<foo>"

In [ ]:
foo_str.strip("<foo>")

## Expresiones regulares (re)

El patrón que buscaremos tiene que coincidir con dos llaves abiertas consecutivas seguidas por cualquier texto ( o no texto) seguido por dos llaves cerradas consecutivas.

In [ ]:
import re

re_string = "{{(.*?)}}"
some_string = "Esta cadena contiene {{palabras}} dentro de {{corchetes}} para mostrar un {{ejemplo}} de {{expresión regular}}"

# findall retorna una lista con las coincidencias en la expresión regular
for match in re.findall(re_string, some_string):
    print("COINCIDENCIAS-> ", match)

In [ ]:
re.findall(re_string, some_string)

Otra manera de trabajar con expresiones regulares en Python es mediante la creación de un objeto de **expresión regular compilada** y utilizar los métodos de ese objeto.

Una **expresión regular compilada** es un objeto que se creó al pasar un patrón a `re.compile()`. Contiene un número de métodos de expresión regular que se crearon al pasar un patrón a `re.compile()`. Existen dos diferencias principales entre utilizar los ejemplos compilados y no compilados. Primero, en lugar de mantener una referencia al patrón de expresión regular "`{{(.*?)}}`" se crea un objeto de expresión regular compilada y se utiliza el patrón para crearlo. En segundo lugar, en lugar de invocar a `findall()` del módulo `re` se invoca `findall()` en el objeto de expresión regular compilada.

In [ ]:
import re

re_obj = re.compile("{{(.*?)}}")
some_string = "Esta cadena contiene {{palabras}} dentro de {{corchetes}} para mostrar un {{ejemplo}} de {{expresión regular}}"

for match in re_obj.findall(some_string):
    print("COINCIDENCIAS-> ", match)

El segundo método (expresión regular compilada) es mucho más eficiente en términos de tiempo de búsqueda por lo que es recomendable su uso en vez de la primera opción.

## Subprocess

Subprocess puede realizar las siguientes operaciones: originar nuevos procesos conectados a entrada salida, conectados a salida estandar, conectados a flujos de error y escuchando código de retorno. Para entender el funcionamiento se aplicará un ejemplo sencillo.

In [ ]:
import subprocess

subprocess.call('df -k', shell=True)

La salida de este comando es un $0$ dado que, al ser un subproceso está saliendo por la salida estandar, que en este caso es la que origina Jupyter, el terminal. Si vemos en el terminal donde se lanzó el cuadernillo vemos la salida del comando. Los códigos de retorno más comunes son:
- 0: Éxito
- 1: Errores generales
- 2: Mal uso de programas internos en shell.
- 126: El comando invocado no se puede ejecutar.
- 127: Comando no encontrado
- 128: Argumento no válido para salid.
- 130: Script terminado por Control-C.

A `subprocess.call()` se le pueden añadir más opciones como:

```python
subprocess.call("ping -c 1 10.0.1.1",
                shell=True,
                stdout=open('/dev/null', 'w'),
                stderr=subprocess.STDOUT)
```

Si se necesita **capturar la salida de un comando** se utiliza `subprocess.Popen`. Otra diferencia importante es que `subprocess.call()` se queda bloqueado esperando una respuesta mientras que `suprocess.Popen()`, no.

In [ ]:
p = subprocess.Popen('df -h', shell=True, stdout=subprocess.PIPE)
out = p.stdout.readlines()

for line in out:
    print(line.strip())

### Nota: *args y *kargs

`*args` y `**kwargs` permiten pasar un número variable de argumentos a una función cuando no sabes de antemano cuántos argumentos puede pasar el usuario a la función, así que en este caso usas estas dos palabras clave cuya diferencia está en que la segunda hace referencia a un diccionario. A continuación, dos ejemplos.

In [ ]:
def test_var_args(f_arg, *argv):
    print "first normal arg:", f_arg
    for arg in argv:
        print "another arg through *argv :", arg

test_var_args('yasoob','python','eggs','test')

In [ ]:
def test_kwargs(**kwargs):
    if kwargs is not None:
        for key, value in kwargs.iteritems():
            print "%s == %s" %(key, value)
 

my_kwargs = {"arg3": 3, "arg2": "two","arg1":5}
greet_me(**my_kwargs)

In [ ]:
def test_kwargs(**kwargs):
    if kwargs is not None:
        for key, value in kwargs.iteritems():
            print "%s == %s" %(key, value)
 
test_kwargs(arg1=3, arg2="two", arg3=5)

Lo que permite las funciones de Python `*args` y `**kwargs` es poder definir funciones donde se le pasarán distintos números de argumentos definiendo una única función. Por ejemplo, ejecutar secuencias de comandos o ejecutar comandos en distintas máquinas podría estar contenido en una función como esta:

In [ ]:
def multi(*args):
    for cmd in args:
        p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        out = p.stdout.read()
        print(out)

También puede comunicarse escribiendo en `stdin`. La utilidad de contar palabras se le dirá que escuche la entrada estándar y luego se escribirá una cadena de caracteres para que procese la utilidad. *Recordar que el resultado sale por la terminal*.

In [ ]:
p = subprocess.Popen("wc -c", shell=True, stdin=subprocess.PIPE)
p.communicate("holaquetal")

Puede hacerse el mismo proceso leyendo de un archivo:

In [ ]:
with open('text.txt', 'w') as file:
    file.write("holaquetal")
    
file = open('text.txt')
f = file.read()

p = subprocess.Popen("wc -c", shell=True, stdin=subprocess.PIPE)
p.communicate(f)

Sería equivalente en bash a:

```bash
wc -c < text.txt

10
```

Para encadenar comandos con Subprocess como haríamos en Bash: `cat /etc/passwd | grep 0:0 | cut -d ':' -f 7` hay que enlazar la salida de uno con la entrada `stdin` del siguiente de la siguiente manera:

In [ ]:
p1 = subprocess.Popen('cat /etc/passwd', shell=True, stdout=subprocess.PIPE)
p2 = subprocess.Popen('grep 0:0', shell=True, stdin=p1.stdout, stdout=subprocess.PIPE)
p3 = subprocess.Popen("cut -d ': ' -f 7", shell=True, stdin=p2.stdout, stdout=subprocess.PIPE)
print(p3.stdout.read())

En este caso no sería necesario concatenera subprocesos para obtener esa información, bastaría con utilizar el módulo de Python `pwd` y hacer un `pwd.getpwnam('root')` para obtener una lista cuyo último elemento es el mismo que todo el proceso realizado con los subproces: `/bin/sh`. Se muestra el proceso de concatenación a modo ilustrativo.

## Hilos de ejecución

Los hilos de ejecución se ejecutan dentro del mismo proceso y comparten estado y memoria a diferencia de los procesos. Como ventaja es que puede crear una estructura de datos a la que pueden acceder todos los hilos de ejecución sin crear un mecanismo de comunicación entre aplicaciones.

Como desventaja, a menudo un programa que sea complejo es dificil de depurar si se encuentra un problema en la ejecución y es necesario añadir un seguimiento (y aún así es complejo).

Veamos un ejemplo sencillo de implementación de hilos. En el script se hereda de la clase `threading.Thread`, se establece una variable `count` y después se anula el método ejecutar para, por último, lanzar cinco hilos de ejecución que explícitamente imprimen su número. El ejemplo es muy simple y con mal diseño ya que se está utilizando una variable global y permite que los hilos puedan compartir estado. Lo ideal sería utilizar colas con hilos de ejecución ya que se ocupan de la complejidad de tratar con estados compartidos (condicionamiento de carrera). De hecho si se ejecuta varias veces puede verse como en algunas ocasiones no se listan los 5 hilos y algunos comparten el valor de la variable `count`. Esto puede ser un problema cuando se quiere trazar.

In [ ]:
# Mal diseño por estado compartido:
import threading
import time

count = 1
class MyThread(threading.Thread):
    def run(self):
        global count
        print("Thread # %s " % count)
        count += 1
        time.sleep(2)
        print("Terminado.\r")
        
for i in range(5):
    MyThread().start()

Nótese como las primeras trazas (Thread n) se lanzan a la vez y pasados dos segundos devuelven el resultado todas a la vez. El método `run` que se ha implementado, si no se hubieran usado hilos iría de manera secuencial.

Veamos ahora un ejemplo utilizando una cola (Queue) y esperar respuetas de ping.

In [ ]:
from threading import Thread
import subprocess
from Queue import Queue

num_threads = 3
queue = Queue()
ips = ['10.0.1.1', '10.0.1.3', '10.0.1.11', '10.0.1.51']

def pinger(i, q):
    '''Ping a una subred'''
    
    while True:
        ip = q.get()
        print("Thread # %s: Pinging %s\n" % (i, ip))
        ret = subprocess.call('ping -c 1 %s' %ip,
                              shell = True,
                              stdout = open('/dev/null', 'w'),
                             stderr=subprocess.STDOUT)
        if ret == 0:
            print("%s is alive" %ip)
        else:
            print("%s: no responde" % ip)
        q.task_done()
        
for i in range(num_threads):
    
    worker = Thread(target=pinger, args=(i, queue))
    worker.setDaemon(True)
    worker.start()

for ip in ips:
    queue.put(ip)
    
print("Hilo principal esperando ...")
queue.join()
print("Hecho!")

Utilizar hilos de ejecución para desarrollar un barrido ping en una subred es un buen ejemplo. Un programa Python sin hilos de ejecución tardaría $N*(tiempo medio de respuesta de ping)$ en resolver todas las IPs. Existen dos estados: estado respuesta y estado *timeout*. Una red típica sería una mezcla entre ambas. 

Esto significa que una red de clase C (254 direcciones) podría llevar unos $3\ segundos$ utilizando hilos de ejecución. Sin hilos podría llevar unos $12.7\ minutos$.

El módulo `Queue` empleado facilita al programador la necesidad de controlar el condicionamiento de carrera y realiza los `mutex` de manera interna, protegiendo las variables para no tener dos accesos en el mismo instante. Queue proporciona a un grupo de "trabajadores" algo que hacer y cuando la cola está vacía se ha acabado el trabajo.

En el código anterior, la parte clave de la cola es el método `join()`. Es una forma de controlar que el hilo de ejecución principal no abandone el programa antes que los otros hilos de ejecución tengan la posibilidad de terminar de trabajar en elementos de una cola. Si comentamos esa línea, el programa principal termina antes que los hilos de ejecución siendo esto un verdadero problema.

Destacable también el método `q.task_done()` al final del método `pinger()` que le dice a la cola que se ha completado la tarea y, por tanto, lo libera de la cola. Así mismo `q.get()` pide a la cola una tarea que realizar.

Es importante también el uso de `setDaemon(True)` antes de invocar al método `start()` para que el programa NO se cuelgue indefinidamente ya que un programa solo saldrá o terminará si se están ejecutando hilos de ejecución *daemon*.

### Programar procesos en Python

Una manera que tiene un administrador de sistemas de enviar un reporte diario sobre el uso de disco (por ejemplo) es programar con la herramienta de UNIX `cron` el envio automático de un correo del siguiente modo:

```bash
59 23 * * * df -h | mail -s "Reporte diario\n" correo@dominio.com
```

La manera en la que se podría hacer con Python es crear un script y que sea lanzado por la herramienta Cron. Un ejemplo de un script de correo sería:

In [ ]:
import smtplib
import subprocess
import string

p = subprocess.Popen("df -h", shell=True, stdout=subprocess.PIPE)
MSG = p.stdout.read()
FROM = "sysadmin@dominio.com"
TO = "correo@dominio.com"
SUBJECT = "Informe diario"
msg = string.join(("From: %s" % FROM,
                  "To: %s" % TO,
                  "Subject: %s" % SUBJECT,
                  "",
                  MSG), "\r\n")
server = smtplib.SMTP('localhost')
server.sendmail(FROM, TO, msg)
server.quit()

### daemonizador

Las tres características que tienen que reunirse para diferenciar un daemon de un proceso en segundo plano son:
- Ejecutarse en segundo plano.
- Estar desplazado del proceso que lo inició.
- No tener terminal de control.

Un proceso en segundo plano solo tendrá una de las tres características mencionadas.

La función `daemonize()` hará que el programa se convierta en un *daemon* en el sentido de las características anteriores. Lo primero que hace este código es crear un `fork()` para crear una copia del proceso que se ejecuta donde la copia se considera el proceso "hijo" y el original se considera como el proceso "padre". Cuando el proceso hijo se bifurca, el padre puede terminar. También se comprueba qué `PID` está detrás de la bifurcación. Si el `PID` es positivo significa que estamos en el proceso padre. Esta es una característica cuando se invoca a un proceso hijo, que el `PID` es negativo. Después de completar la llamada a `os.fork()` habrá dos copias del mismo proceso ejecutándose. Ambos luego comprueban el código de retorno de la llamada `fork()` que devuelve 0 en el hijo y el ID del proceso en el padre. **Cualquiera que sea el proceso que tiene un código de retorno `!= 0` que solamente será del padre, termina**. Si ocurre un problema en este punto, se eleva una excepción y el proceso termina. Si el script se lanza desde una Bash se verá que se devuelve el *command prompt*.

Lo siguiente que hace el proceso es cambiar el directorio a `/`, establecer su `umask` en 0 y crear una nueva sesión.  Cambiar el directorio sitúa al proceso en un directorio que siempre terminará. La última acción (`os.setsid()`) es menos familiar pero realiza, en primer lugar, que el proceso se convierta en líder de sesión de una nuev sesión. A continuación hace que el proceso se convierta en líder de grupo del proceso de un nuevo grupo de proceso. Por último hace que el proceso no tenga terminal de control, lo que significa que el proceso no puede ser víctima de acciones involuntarias de control de trabajo desde algún terminal. Esto es útil para tener un proceso prolongado e ininterrumpido como un daemon.

In [7]:
import sys, os

def daemonize(stdin='/dev/null', stdout='/dev/null', stderr='/dev/null'):
    # Realizar primero fork.
    try:
        pid = os.fork()
        if pid > 0:
            sys.exit(0)
    except OSError, e:
        sys.stderr.write("fork #1 failed: (%d) %s\n" % (e.errno, e.strerror))
    # Separarde entorno padre.
    os.chdir("/")
    os.umask(0)
    os.setsid()
    # Realizar segundo fork.
    try:
        pid = os.fork()
        if pid > 0:
            sys.exit(0)
    except OSError, e:
        sys.stderr.write("fork #2 failed: (%d) %s\n" % (e.errno, e.strerror))
    sys.exit(1)
        
    # El proceso ahora está daemonizado
    for f in sys.stdout, sys.stderr: f.flush()

    si = file(stdin, 'r')
    so = file(stdout, 'a+')
    se = file(stderr, 'a+', 0)

    os.dup2(si.fileno( ), sys.stdin.fileno( ))
    os.dup2(so.fileno( ), sys.stdout.fileno( ))
    os.dup2(se.fileno( ), sys.stderr.fileno( ))

Si guardamos este código en un archivo llamado `daemonize.py` tenemos la estructura preparada para crear un programa que haga uso de él como puede ser el siguiente:

In [8]:
#from daemonize import daemonize
import time
import sys

def mod_5_watcher():
    start_time = time.time()
    end_time = start_time + 20
    
    while time.time() < end_time:
        now = time.time()
        if int(now) % 5 == 0:
            sys.stderr.write('Mod 5 at %s\n' % now)
        else:
            sys.stdout.write('No mod 5 at %s\n' % now)
        time.slee(1)

if __name__ == "__main__":
    daemonize(stdout='/tmp/stdout.log', stderr='/tmp/stderr.log')
    mod_5_watcher()

SystemExit: 0

El script primero *daemoniza* y especifica que se debería utilizar `/tmp/stdout.log` para la salida estandar y `/tmp/stderr.log` para error estándar. Luego se observa el tiempo durante los siguientes 20 segundos, durmiendo un segundo entre comprobaciones de tiempo. Si el tiempo es divisible entre 5, se escribe el error en estándar.Puesto que el proceso utiliza `/tmp/stdout.log` para salida estándar y `/tmp/stderr.log` para error estándar se debería poder ver los resultados en esos archivos después de ejecutar el ejemplo.

## Persistencia de datos

### Pickle

Es el mecanísmo más básico de serialización o almacenamiento de datos sin relación y está incluido en la biblioteca estándar. La idea del módulo `pickle` (conservar) es coger un objeto, escribirlo en disco, salir del proceso y regresar más tarde para iniciar de nuevo el proceso, leere el objeto de disco e interactuar con él. Los tipos de objetos que puede preservar se pueden resumir en:
- `None`, `True`, `False`.
- Enteros, con coma flotante, complejos, enteros largos.
- Cadenas normales y Unicode.
- Tuplas, listas, conjuntos y diccionarios que contienen objetos que se pueden conservar.
- Funciones definidas en el nuvel superior de un módulo.
- Clases que se definen en el nivel superior del módulo.
- Instancias de dichas clases cuyos `__dict__` o `__setstate__()` se pueden conservar.

Un ejemplo de la librería Pickle es el siguiente:

In [2]:
import pickle

diccionario = {'a': 1, 'b': 2}

pickle_file = open('pickle_test.pkl', 'wb')
pickle.dump(diccionario, pickle_file)
pickle_file.close()

*Nota: fijarse en que el archivo tiene que escribirse en formato `wb`, es decir, en formato binario*.

Si abrimos el archivo `.pkl` generado 

In [3]:
cat pickle_test.pkl

�}q (X   aqKX   bqKu.

Para comprobar la persistencia, se carga el archivo `pkl` y se comprueba su contenido. *De igual manera, al haber sido almacenado en formato "bytes" tiene que cargarse de igual manera* (`rb`).

In [6]:
import pickle

pickle_file = open('pickle_test.pkl', 'rb')

diccionario_cargado = pickle.load(pickle_file)

print(diccionario_cargado)

{'a': 1, 'b': 2}


Es interesante recalcar que no es necesaria una relación 1 a 1 entre los objetos y los archivos de Pickle. Se pueden volcar tantos objetos a un solo archivo Pickle como espacio en disco se tenga. Un ejemplo de cómo volcar un número de objetos (diccionarios) a un archivo Pickle sería:

In [10]:
import pickle

lista_dicts = [{str(i): i} for i in range(5)]
print(lista_dicts)

pickle_file = open('list_of_dicts.pkl', 'wb')

for d in lista_dicts:
    pickle.dump(d, pickle_file)
    
pickle_file.close()

[{'0': 0}, {'1': 1}, {'2': 2}, {'3': 3}, {'4': 4}]


Es el mismo ejemplo que se ha utilizado en el ejemplo anterior pero llamando varias veces a `dump()` para almacenar todos los diccionarios. Si de igual manera cargamos el contenido del archivo `pkl` veremos que se leen y cargan en el mismo orden que fueron almacenados (FIFO) y con el mismo contenido.

In [12]:
import pickle

pickle_file = open('list_of_dicts.pkl', 'rb')

while 1:
    try:
        print(pickle.load(pickle_file))
    except EOFError:
        print("Fin de fichero")
        break

{'0': 0}
{'1': 1}
{'2': 2}
{'3': 3}
{'4': 4}
Fin de fichero


Como se ha dicho, Pickle permite también almacenar otros tipos de datos como **clases**. Tomando una clase como ejemplo vamos a ver el mismo procedimiento, crear, almacenar, cargar y ejecutar:

In [15]:
import pickle

class MyClass(object):
    def __init__(self):
        self.data = []
    def __str__(self):
        return "Clase personalizada: %s" % str(self.data)
    def add_item(self, item):
        self.data.append(item)
        

my_obj = MyClass()
my_obj.add_item(1)
my_obj.add_item(2)
my_obj.add_item(3)

print(my_obj)

pickle_file = open('pickle_class.pkl', 'wb')
pickle.dump(my_obj, pickle_file)
pickle_file.close()

Clase personalizada:: [1, 2, 3]


Si cargamos el archivo y ejecutamos tenemos:

In [17]:
import pickle

pickle_file = open('pickle_class.pkl', 'rb')

my_obj = pickle.load(pickle_file)
print(my_obj)

pickle_file.close()

Clase personalizada:: [1, 2, 3]


### YAML

Para algunos *YAML ain't markup language* para otros *yet another markup language*, YAML es un formato de datos que se utiliza a menudo para almacenar, recuperar y actualizar datos en un diseño de texto sencillo. Estos datos a menudo son jerárquicos. Una razón para trabajar con YAML en vez de con Pickle es que YAML es legible. La sintaxis es similar a un archivo de configuración. Si existe un caso donde haya que editar un archivo de configuración es una buena elección. Otra razón es que lo analizadores YAML se han implementado en muchos otros lenguajes (como JSON). Si se necesita obtener datos entre una aplicación escrita en Python y otro lenguaje, YAML puede ser una buena solución intermedia. Veamos un ejemplo:

In [37]:
import yaml

yaml_file = open('test.yml', 'w')

diccionario = {'foo': 'a', 'bar': 'b', 'bam': [1, 2, 3]}

yaml.dump(diccionario, yaml_file, default_flow_style=False)
yaml_file.close()

La opción `default_flow_syle` desactiva serializar los datos almacenados en modo bloque. Puede ejecutarse la celdilla cambiando a `True` la bandera para ver la diferencia. Si ahora vemos el contenido del archivo con `cat` tenemos:

In [39]:
cat test.yml

bam:
- 1
- 2
- 3
bar: b
foo: a


Y si cargamos el contenido como en los ejemplos con Pickle, se tiene:

In [30]:
import yaml

yaml_file = open('test.yml', 'r')

yaml.load(yaml_file, Loader=yaml.FullLoader)

{'bam': [1, 2, 3], 'bar': 'b', 'foo': 'a'}

El método de carga de yaml, `load`, está *deprecated*. En su lugar se tiene que usar:
- `yaml.safe_load`
- `yaml.full_load`
- `yaml.unsafe_load`

O puede usarse a través del método `load` especificando el tipo de carga:

```pythoon
yaml.load(input, Loader=yaml.FullLoader)
```

Donde las opciones `Loader` disponibles son:

- `BaseLoader`: Carga básica de un archivo YAML:
- `SafeLoader`: Carga un subconjunto del lenguaje YAML, de forma segura. Es recomendable para cargar datos no confiables.
- `FullLoader` : Carga el lenguaje YAML completo. Evita la ejecución arbitraria de código. Este es actualmente (PyYAML 5.1) el cargador por defecto llamado por yaml.load(input) (después de emitir la advertencia).
- `UnsafeLoader`:  También llamado `Loader` de retrocompatibilidad. El código original del cargador que podría ser fácilmente explotable por la entrada de datos no confiables.

Si queremos mantener una estructura en el almacenamiento de los datos, podemos dejar fuera la opción del archivo en la llamada a `dump()` y en lugar de guardarse, imprimirá por la salida estandar el resultado, como puede verse en el siguiente ejemplo.

In [51]:
import yaml

diccionario = {'first': {'second': {'third': {'fourth': 'a'}}}}
print("Diccionario 1:")
print(yaml.dump(diccionario, default_flow_style=False))

diccionario_2 = [{'a': 'a'}, {'b': 'b'}, {'c': 'c'}]
print("Diccionario 2:")
print(yaml.dump(diccionario_2, default_flow_style=False))

diccionario_3 = [{'a': 'a'}, {'b': 'b'}, {'c': [1, 2, 3, 4, 5]}]
print("Diccionario 3:")
print(yaml.dump(diccionario_3, default_flow_style=False))
print("Diccionario 3 con el estilo por defecto:")
print(yaml.dump(diccionario_3, default_flow_style=True))

Diccionario 1:
first:
  second:
    third:
      fourth: a

Diccionario 2:
- a: a
- b: b
- c: c

Diccionario 3:
- a: a
- b: b
- c:
  - 1
  - 2
  - 3
  - 4
  - 5

Diccionario 3 con el estilo por defecto
[{a: a}, {b: b}, {c: [1, 2, 3, 4, 5]}]



### Otras librerías

Existen otras librerías para persistencia de datos como son:

- **cPicke**: Muy parecida a Picke pero implementada en C.
- **Shelve**: Proporciona una interfaz sencilla para persistencia de objetos y simplifica la persistencia de múltiples objetos.
- **ZODB**: Es similar a YAML o Pickle pero con la posibilidad de escalar bajo nuestras necesidad propocionando atomicidad en las operaciones y transacciones. Es de tipo 'persistencia relacional'.
- **SQLite**: De tipo 'persistencia relacional' permite tratar el almacenamiento de los datos desde cero y el motor de la base de datos está, en vez de un servidor externo, ejecutándose en el mismo proceso que el código y se accede a él como una biblioteca más. Se guarda todo en un archivo, es compatible con muchos otros lenguajes de programación y, puede decirse, que hará las veces de base de datos "real".
- **ORB Storm**: Permite acceso a los datos de manera sencilla gracias al mapeado objeto-relacional. Simplemente es proporcionar una interfaz orientada a objetos a la base de datos.
- **SQLite-orm**: Una librería que comunica SQLite con el programa utilizando ORM con mantenimiento hasta 2018 (Storm dejó de ser actualizada en 2013).
